In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from transformers import T5Tokenizer, T5Model,T5ForConditionalGeneration, pipeline
import math
import time
import random

In [4]:
!pwd

/scratch/joshi.at/ppda_project


In [120]:
df = pd.read_csv('ytdataset_preprocessed.csv')

In [123]:
df = df.drop(columns = ['Unnamed: 0'])

In [121]:
df.head()

,Unnamed: 0,title,transcript,playlist_name
0,0,introduction to vertex ai feature store,priyanka vergadia did you know that most of th...,ai and machine learning with google cloud
1,1,illuminating the global fishing fleet through ...,speaker oceanic ecosystems are threatened by ...,ai and machine learning with google cloud
2,2,hyperparameter tuning on vertex ai,priyanka vergadia hi im priyanka vergadia and ...,ai and machine learning with google cloud
3,3,endtoend mlops with vertex ai,priyanka vergadia one of the biggest challenge...,ai and machine learning with google cloud
4,4,transformers explained understand the model be...,so if you remember anything about transformers...,the original transformer was designed for tran...


In [11]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

/home/joshi.at/.conda/envs/ath_pytorch/lib/python3.7/site-packages/transformers/models/t5/tokenization_t5.py:173: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [33]:
input_ids = tokenizer.encode(tokenizer.decode(chunked_sample[0]), return_tensors='pt')

In [22]:
torch.tensor(chunked_sample[0])

tensor([    3, 15582,   476,  5033, 12048,     3, 16174,  6302, 24605,    10,
         3963,    25,   214,    24,   167,    13,     8,    97,  1869,    57,
          331,  7004,  1550,   139,     3,   210,  6287,   697,   331,    58,
         1537,  3346,     6,    16,  1451,  3867,     6,    84,    19,     3,
        21139,  5902,   331,   139,   306,    18,  4497,  3785,  9650,    21,
            3,  6858,  2250,     5,   299,    48,   433,    19,   557,    16,
        16995,    11,     3,  2160,  8692,     5,  1548,    27,    31,    51,
            3,  7855,    63,  5979,     9,     6,    11,    16,    48,   671,
           62,    56,  2862,     8,   843,  2428,    28,  1451,  3867,     6,
          149,   781, 10354,     3, 16772,  4493,   199,  4602,   135,     6,
           11,   217,     3,     9,  1704,  8698,     5,   852,   125,    33,
            8,   843,  2428,    28,     3,  6858,   753,    58,    37,   166,
           19,    24,    79,    33,   614,    12,   698,    11, 

In [32]:
tokenizer.decode(chunked_sample[0])

"PRIYANKA VERGADIA: Did you know that most of the time spent by data scientists goes into wrangling data? More specifically, in feature engineering, which is transforming raw data into high-quality input signals for ML models. But this process is often inefficient and brittle. Well I'm Priyanka, and in this video we will identify the key challenges with feature engineering, how Vertex Feature Store help solve them, and see a quick demo. Now what are the key challenges with ML features? The first is that they are hard to share and reuse across your different steps of the ML workflow and across projects, which results in duplicate efforts. Second is that it is hard to serve in production reliably with lower latency, and the third is that there is an inadvertent skew in feature values between training and serving, usually, which causes your model quality to degrade over time. That is exactly where Vertex Feature Store comes in. It's a fully managed and unified solution to share, discover,

In [34]:
summary_ids = model.generate(input_ids,
                            min_length=100, max_length=500,temperature=0.7
                            )
# Decode the summary text using the tokenizer
summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
# Print the summary text
print(summary_text)

Priyanka VERGADIA: feature engineering is a time-consuming and inefficient process. Feature Store helps you share, discover, and serve machine learning features at scale. it's a fully managed and unified solution to share, discover, and serve ML features. Feature Store is a fully managed and unified solution to share, discover, and serve machine learning features at scale. Priyanka: 'i'm a big fan of this product, but i


[{'summary_text': "Priyanka VERGADIA shows how vertex Feature Store helps solve key challenges . it's a fully managed and unified solution to share, discover, and serve machine learning features . Feature store makes the features reusable, easy to serve, and avoids skew ."}]

In [10]:
tokenizer = T5Tokenizer.from_pretrained('t5-base',return_tensors='pt')

/home/joshi.at/.conda/envs/ath_pytorch/lib/python3.7/site-packages/transformers/models/t5/tokenization_t5.py:173: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [12]:
len(tokenizer(df['transcript'][0]).input_ids)

Token indices sequence length is longer than the specified maximum sequence length for this model (1312 > 512). Running this sequence through the model will result in indexing errors


1312

In [14]:
model = T5Model.from_pretrained('t5-base')

In [13]:
chunked_sample = np.array_split(tokenizer(df['transcript'][0]).input_ids,3)

In [14]:
len(chunked_sample[2])

437

In [36]:
# define model, tokenizer and summarization pipeline
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')
summarizer_gpu = pipeline("summarization", model="t5-base", tokenizer="t5-base", device=0)
summarizer_cpu = pipeline("summarization", model="t5-base", tokenizer="t5-base")

/home/joshi.at/.conda/envs/ath_pytorch/lib/python3.7/site-packages/transformers/models/t5/tokenization_t5.py:173: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,
/home/joshi.at/.conda/envs/ath_pytorch/lib/python3.7/site-packages/transformers/models/t5/tokenization_t5_fast.py:165: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will b

In [94]:
def create_summaries(transcript,device,time_store,MAX_LENGTH=500):
    full_tokenize = tokenizer(transcript).input_ids
    length_sample = len(full_tokenize)
    chunks =math.ceil(length_sample/MAX_LENGTH)
    chunked_sample = np.array_split(full_tokenize,chunks)
    sample_text=""
    srz_min_length = 1    # to be considered
    srz_max_length = int(MAX_LENGTH/chunks)
    if (device==0):
        summarizer = summarizer_gpu
    else:
        summarizer = summarizer_cpu
    start = time.time()
    for j in chunked_sample:
        input_text = tokenizer.decode(j)
        sample_text+=summarizer(input_text, min_length=srz_min_length, max_length=srz_max_length)[0]['summary_text']
    end = time.time()
    time_store.append(end-start)
    return sample_text

In [86]:
script = df['transcript'][3]
temp=[]
device=0
summary = create_summaries(script,device,temp)

In [87]:
temp

[7.478700637817383]

In [21]:
chunkedtext = tokenizer.decode(chunked_sample[0])

3

In [ ]:
chunkedtext = tokenizer.decode(chunked_sample[0])

In [26]:
a=""
a += "lol"

In [28]:
int(500/3)

166

In [49]:
(8*100)/60

13.333333333333334

In [46]:
df.shape

(5001, 3)

In [ ]:
df['a'] = df['a'].apply(lambda x: x + 1)

,title,transcript,playlist_name
0,introduction to vertex ai feature store,priyanka vergadia did you know that most of th...,ai and machine learning with google cloud
1,illuminating the global fishing fleet through ...,speaker oceanic ecosystems are threatened by ...,ai and machine learning with google cloud
2,hyperparameter tuning on vertex ai,priyanka vergadia hi im priyanka vergadia and ...,ai and machine learning with google cloud
3,endtoend mlops with vertex ai,priyanka vergadia one of the biggest challenge...,ai and machine learning with google cloud
4,transformers explained understand the model be...,so if you remember anything about transformers...,the original transformer was designed for tran...
...,...,...,...
96,using tensorflow hub for more productive machi...,yufeng guo code reuse is a central tenet of so...,where we explore the art
97,training image amp text classification models ...,NaN,ai and machine learning with google cloud
98,deep learning vm images,yufeng guo dont you just love setting up your ...,ai and machine learning with google cloud
99,how to import a keras model into tensorflowjs,yufeng guo tensorflowjs is awesome keras is aw...,and if you enjoyed it


In [92]:
def subset_summary(df, k, device):
    temp = df.loc[random.sample(range(df.shape[0]),k)]
    store=[]
    temp['summary'] = temp['transcript'].apply(lambda x:create_summaries(x,device,store))
    temp = temp.reset_index().drop(['index'],axis=1)
    temp['time_taken'] = store
    return temp

In [62]:
df.loc[random.sample(range(df.shape[0]),3)]

,title,transcript,playlist_name
1082,deep learningcs lec continuous bag of words m...,so from here on so none of this that we covere...,deep learning
1297,tensorflowjs explore tensor operations throug...,whats up guys in this video were going to expl...,keras python deep learning neural network api
2797,cmu advanced nlp language modeling and neura...,okay um this time im going to be talking about...,cmu advanced nlp


In [124]:
new_df = subset_summary(df,10,0)

/home/joshi.at/.conda/envs/ath_pytorch/lib/python3.7/site-packages/transformers/pipelines/base.py:1073: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,
Your max_length is set to 500, but you input_length is only 79. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=39)
Your max_length is set to 500, but you input_length is only 181. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=90)


In [126]:
new_df.time_taken.mean()

6.73138964176178

In [106]:
(12*5000)/3600

16.666666666666668

In [107]:
contains_non_string = df[df['transcript'].apply(type) != str].any()

In [108]:
contains_non_string

title             True
transcript       False
playlist_name     True
dtype: bool

In [122]:
df[df['transcript'].apply(type)!=str]

,Unnamed: 0,title,transcript,playlist_name
16,16,recovering global wildlife populations using ml,NaN,ai and machine learning with google cloud
33,33,integrate dialogflow with google chat,NaN,ai and machine learning with google cloud
63,63,train and deploy models in cloud and onprem fr...,NaN,ai and machine learning with google cloud
80,80,reusable execution in production using papermi...,NaN,ai and machine learning with google cloud
87,87,optimizing tensorflow models for serving googl...,NaN,ai and machine learning with google cloud
...,...,...,...,...
4945,4945,data augmentation explained,NaN,deep learning fundamentals intro to neural ne...
4947,4947,convolutional neural networks cnns explained,NaN,deep learning fundamentals intro to neural ne...
4949,4949,visualizing convolutional filters from a cnn,NaN,deep learning fundamentals intro to neural ne...
4964,4964,finetuning a neural network explained,NaN,deep learning fundamentals intro to neural ne...


In [115]:
df = df[df['transcript'].notna()]

In [119]:
new_df.to_csv('sample_summarized.csv')

In [125]:
new_df

,title,transcript,playlist_name,summary,time_taken
0,neural networks pt relu in action,some people say i mispronounce value but that ...,machine learning,the relu activation function is one of the mos...,6.519664
1,braintobrain communication is coming,dear fellow scholars this is two minute papers...,two minute papers,braintobrain interface uses eeg to record brai...,4.811749
2,how to add a custom ner pipe in spacy and a cu...,hello and welcome back to the series on custom...,spacy for digital humanities with python tutor...,in this video im going to show you how to crea...,8.473709
3,matplotlib tutorial conclusion,what is going on everybody welcome to part of...,matplotlib tutorial series graphing in python,the wireframe is a slightly more complex versi...,6.793439
4,machine learning tutorial python logistic r...,this is part two of logistic regression tutori...,machine learning tutorial python machine lear...,this is part two of the logistic regression tu...,10.672934
5,neural network architectures amp deep learning,welcome back so were talking about neural netw...,intro to data science,the basic building block of a neural network i...,9.124617
6,equation for precision intro to machine learning,so heres my final question which will be chall...,intro to machine learning,click on the corresponding items on the left s...,0.916600
7,what is wordvec a simple explanation deep lea...,word to work is a technique in computer scienc...,deep learning with tensorflow keras and python,word to work is a technique in computer scienc...,13.923655
8,barack obama intro to deep learning mit s,this year i figured we could do something a li...,mit s introduction to deep learning,mit s is the official introductory course on d...,1.451634
9,accelerated computer vision convolutions fil...,so after learning to stand in your networks or...,computer vision,convolutional neural networks are a special ki...,4.625894
